In [ ]:
# # Mount GDrive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# import os

# # os.chdir allows you to change directories, like cd in the Terminal
# os.chdir('/content/drive/MyDrive/Colab Notebooks/sign-language-interpretation')

In [2]:
import matplotlib.pyplot as plotter_lib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL as image_lib
import tensorflow as tflow
from tensorflow.keras.layers import Flatten
from tensorflow.keras import models
from tensorflow.keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.preprocessing import image
from tensorflow.keras.layers import Input, Lambda ,Dense ,Flatten ,Dropout
%matplotlib inline
import os
from tensorflow.keras.callbacks import EarlyStopping
from PIL import Image, ImageOps

2023-05-27 13:57:46.763791: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-27 13:57:47.418462: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-27 13:57:47.425106: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 13:57:51.960786: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def load_image_dataset(root_directory, image_size, batch_size, validate_size):
    # Set the intial DataFrame map
    # letters = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'I':8, 'K':9, 'L':10, 'M':11, 'N':12,
    #             'O':13, 'P':14, 'Q':15, 'R':16, 'S':17, 'T':18, 'U':19, 'V':20, 'W':21, 'X':22, 'Y':23}

    # Create a data generator
    datagen = tflow.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=validate_size)

    # Generate the training dataset from directory using the data generator
    train_generator = datagen.flow_from_directory(
        root_directory,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        shuffle=True)
    
    # Generate the validation dataset from directory using the data generator
    validate_generator = datagen.flow_from_directory(
        root_directory,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=True)
    
    return train_generator, validate_generator

root_directory = os.path.join("raw_data", "landmark_train_data")  # Replace with the root directory path
image_size = (200, 200)
batch_size = 32
validate_size = 0.3   # Set =0 if NO validation data output 

train_data, val_data = load_image_dataset(root_directory, image_size, batch_size, validate_size)

Found 13440 images belonging to 24 classes.
Found 5759 images belonging to 24 classes.


In [16]:
root_directory = os.path.join("raw_data", "landmark_train_data")  # Replace with the root directory path
image_size = (100, 100)
batch_size = 32
validate_size = 0.3   # Set =0 if NO validation data output 

train_data_100, val_data_100 = load_image_dataset(root_directory, image_size, batch_size, validate_size)

Found 13440 images belonging to 24 classes.
Found 5759 images belonging to 24 classes.


In [8]:
train_data.class_indices,  val_data.class_indices

({'A': 0,
  'B': 1,
  'C': 2,
  'D': 3,
  'E': 4,
  'F': 5,
  'G': 6,
  'H': 7,
  'I': 8,
  'K': 9,
  'L': 10,
  'M': 11,
  'N': 12,
  'O': 13,
  'P': 14,
  'Q': 15,
  'R': 16,
  'S': 17,
  'T': 18,
  'U': 19,
  'V': 20,
  'W': 21,
  'X': 22,
  'Y': 23},
 {'A': 0,
  'B': 1,
  'C': 2,
  'D': 3,
  'E': 4,
  'F': 5,
  'G': 6,
  'H': 7,
  'I': 8,
  'K': 9,
  'L': 10,
  'M': 11,
  'N': 12,
  'O': 13,
  'P': 14,
  'Q': 15,
  'R': 16,
  'S': 17,
  'T': 18,
  'U': 19,
  'V': 20,
  'W': 21,
  'X': 22,
  'Y': 23})

In [17]:
def initiliaze_model_vgg16(image_width, image_height, no_chans, n_classes):
    
    #Initialising vgg16 
    classifier_vgg16 = VGG16(input_shape= (image_width,image_height,no_chans),include_top=False,weights='imagenet')
    
    #Don't train existing weights for vgg16
    for layer in classifier_vgg16.layers:
        layer.trainable = False
    
    #Add other layers for pre-trained model    
    classifier = classifier_vgg16.output #head mode
    classifier = Flatten()(classifier) #adding layer of flatten
    classifier = Dense(units=512, activation='relu')(classifier)
    classifier = Dense(units=256, activation='relu')(classifier)
    classifier = Dense(units=128, activation='relu')(classifier)
    classifier = Dropout(0.6)(classifier)
    classifier = Dense(units=n_classes, activation='softmax')(classifier)

    #Build & compile the model
    model = Model(inputs =classifier_vgg16.input, outputs = classifier)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [19]:
gpu_devices = tflow.config.list_physical_devices('GPU')
gpu_devices

[]

In [9]:
es = EarlyStopping(patience=5, restore_best_weights=True)

model_vgg16_100 = initiliaze_model_vgg16(100, 100, 3, 24)

epochs = 10

history_vgg16_100 = model_vgg16_100.fit(
                    train_data_100,
                    validation_data = val_data_100,
                    epochs=epochs,
                    callbacks=[es],
                    verbose=1)

NameError: name 'classifier_vgg16' is not defined

In [18]:
def initialize_model_resn50(image_width, image_height, no_chans, n_classes):
    #Initialising ResNet50 
    classifier_resnet = tflow.keras.applications.ResNet50(input_shape= (image_width,image_height,no_chans),include_top=False,weights='imagenet')
    
    #don't train existing weights for resnet50
    for layer in classifier_resnet.layers:
        layer.trainable = False
     
    #Add the necessary layers for pre-trained model   
    classifier = classifier_resnet.output #head mode
    classifier = Flatten()(classifier) #adding layer of flatten
    classifier = Dense(units=512, activation='relu')(classifier)
    classifier = Dense(units=256, activation='relu')(classifier)
    classifier = Dense(units=128, activation='relu')(classifier)
    classifier = Dropout(0.2)(classifier)
    classifier = Dense(units=n_classes, activation='softmax')(classifier)

    #Build and compile the model
    model = Model(inputs = classifier_resnet.input , outputs = classifier)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
es = EarlyStopping(patience=5, restore_best_weights=True)

model_resn50_100 = initialize_model_resn50(100, 100, 3, 24)

epochs = 10

history_resn50_200 = model_resn50_100.fit(
                    train_data_100,
                    validation_data = val_data_100,
                    epochs=epochs,
                    callbacks=[es],
                    verbose=1)